In [5]:
import os
import re
import json
from sklearn.preprocessing import MultiLabelBinarizer 
import itertools
import pandas as pd
import numpy as np
import webbrowser
from datetime import date
from importlib import reload
from cleaning import *
reload(cleaning)


ImportError: attempted relative import with no known parent package

In [10]:
print(os.chdir(".."))

None


In [ ]:
df = load_and_prepare_data(location="notebook")


In [ ]:
str(pd.to_datetime('today').date())

In [ ]:
df[
    (df.preis.isna()==0)
    &(df.suchOrt=="norderstedt")
    &(df.transaktionsArt=="mieten")
    &(df["datumDownload"]=='2021-04-24')#pd.to_datetime('today').normalize())
]

In [ ]:
interessanteCols = ["preis","anzahl_raeume","wohnflaeche","transaktionsArt","url","datumDownload","plz"]


In [ ]:
mask = ((df["anzahl_raeume"]>=3) 
        #& (df["wohnflaeche"]>100) 
        & (df["transaktionsArt"]=="mieten") 
        #& (df["preis"]<1500)
        & (df["datumDownload"]==pd.to_datetime('today').normalize())
        #& (df["plz"]!="24568")
       )
(df.loc[mask,interessanteCols].sort_values(by=["datumDownload"],ascending=False))

In [ ]:
mask = ((df["anzahl_raeume"]>=4) 
        & (df["wohnflaeche"]>120) 
        & (df["transaktionsArt"]=="kaufen") 
        & (df["preis"]<500000)
        & (df["datumDownload"]==pd.to_datetime('today').normalize())
       # & (df["plz"]!="24568")
        #& (df["plz"]!="29493")
       )
subsel = (df.loc[mask,interessanteCols]
 .sort_values(by=["preis","anzahl_raeume"])
)
subsel

In [ ]:

firefoxPath = "/Applications/Firefox.app/Contents/MacOS"
client = webbrowser.get("open -a /Applications/Firefox.app %s")
for url in subsel.url:
    client.open(url)